# Importing required libraries

In [1]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Indeed.com

In [2]:
def extract_Indeed(page):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    url=f"https://in.indeed.com/jobs?q=data%20scientist&l=India&start={page}"
    r=requests.get(url,headers)
    soup=BeautifulSoup(r.content,'html.parser')
    return soup

In [3]:
def transform_Indeed(soup):
    divs=soup.find_all('div',class_='job_seen_beacon')
    for item in divs:
        title=item.find('h2').text.strip()
        title=title.split('new')
        if len(title)>1:
          title=title[1]
        else:
          title=title[0]
        company=item.find('span',class_='companyName').text.strip()
        location=item.find('div',class_='companyLocation').text.strip()
        date=item.find('span',class_='date').text.strip()
        Indeed_jobs={'title':title,'company':company,'location':location,'dateposted':date}
        job.append(Indeed_jobs)
    return

# Linked In

In [4]:
def extract_LinkedIn(page):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    url=f'https://www.linkedin.com/jobs/search/?geoId=102713980&keywords=data%20scientist&start={page}'
    r=requests.get(url,headers)
    soup1=BeautifulSoup(r.content,'html.parser')
    return soup1

In [5]:
def transform_LinkedIn(soup1):
    divs=soup1.find_all('div',class_='base-card base-card--link base-search-card base-search-card--link job-search-card')
    for item in divs:
        title=item.find('a').text.strip()
        company=item.find('a',class_="hidden-nested-link").text.strip()
        location=item.find('span',class_="job-search-card__location").text.strip()
        date=item.find('time').text.strip()
        linkedinjobs={'title':title,'company':company,'location':location,'dateposted':date}
        job1.append(linkedinjobs)
    return

# Shine.com

In [6]:
def extract_Shine(page):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}
    url=f'https://www.shine.com/job-search/data-scientist-jobs-={page}?q=Data%20Scientist'
    r=requests.get(url,headers)
    soup2=BeautifulSoup(r.content,'html.parser')
    return soup2 

In [7]:
def transform_Shine(soup2):
    divs=soup2.find_all('div',class_='ParentClass')
    for item in divs:
        job=item.find_all('h2')
        for i in job:
            title.append(str(i).split('>')[1].split('<')[0])
        comp=item.find_all('div',class_="jobCard_jobCard_cName__mYnow")
        for j in comp:
            company.append(str(j).split('<span>')[1].split('<')[0])
        loc=item.find_all('div',class_="jobCard_jobCard_lists_item__YxRkV jobCard_locationIcon__zrWt2")
        for k in loc:
            location.append(str(k).split('>')[1].split('<')[0])
        posted=item.find_all('div',class_="jobCard_jobCard_features__wJid6")
        for l in posted:
            dateposted.append(str(l).split('<span>')[1].split('<')[0])    
    return

# Execution

In [8]:
start=time.time()
job=[]
job1=[]
title=[]
company=[]
location=[]
dateposted=[]


#indeed
for i in range(0,200,10):
    c=extract_Indeed(i)
    transform_Indeed(c)   
Indeed=pd.DataFrame(job)


#linkedin
for i in range(0,200,25):
    c1=extract_LinkedIn(i)
    transform_LinkedIn(c1)
LinkedIn=pd.DataFrame(job1)

#shine.com
for i in range(1,30,1):
    c=extract_Shine(0)
    transform_Shine(c)
    data=list(zip(title,company,location,dateposted))
df_shine=pd.DataFrame(data,columns=['title','company','location','dateposted'])

In [9]:
final_df=pd.concat([Indeed,LinkedIn,df_shine])
print('Combined entries size=',len(final_df))
final_df=final_df.reset_index(drop=True)
#checking for duplicate entries and deleting them
print('Duplicate entries found =' ,len(final_df[final_df.duplicated()]))
final_df=final_df.drop_duplicates()
final_df=final_df.reset_index(drop=True)
end=time.time()
print('Total unique entries =',len(final_df))

Combined entries size= 1064
Duplicate entries found = 721
Total unique entries = 343


In [10]:
print("Total time of execution =",round(end-start,5),'seconds')

Total time of execution = 59.1785 seconds


In [11]:
final_df.head()

,title,company,location,dateposted
0,Data Scientist,HP,"Bengaluru, Karnataka",Posted1 day ago
1,Data Scientist,"JPMorgan Chase Bank, N.A.","Bengaluru, Karnataka",PostedJust posted
2,Data Scientist,Codvo.ai,Remote,PostedToday
3,Data Scientist,MedTourEasy,Remote,PostedToday
4,Data Scientist – AI / Deep Learning for Automa...,Bosch Group,"Bengaluru, Karnataka",PostedJust posted


In [13]:
final_df.to_csv(r'C:\Users\erict\Desktop\Aivamastu\Jobs_extracted.csv',index=False)

# Shortcomings 

* All data are from the job listings for data scientist(result page), could'nt figure out how to extract data from each job listing individually.